### Importing neccessary libraries

In [1]:
import numpy as np
import os
import string
from PIL import Image
from pickle import dump
from pickle import load
from keras.applications.xception import Xception #to get pre-trained model Xception
from keras.applications.xception import preprocess_input
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from keras.preprocessing.text import Tokenizer #for text tokenization
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Add
from keras.utils import to_categorical
from keras.models import Model, load_model
from keras.layers import LSTM, Embedding, Dropout
from keras.layers import Input, Dense #Keras to build our CNN and LSTM
from tqdm import tqdm

### Data Cleaning

In [2]:
import string

def load_fp(filename):
    with open(filename, 'r') as file:
        return file.read()

def extract_captions(filename):
    data = load_fp(filename)
    captions = data.split('\n')
    descriptions = {}
    for caption in captions[:-1]:
        img, caption = caption.split('\t')
        descriptions.setdefault(img[:-2], []).append(caption)
    return descriptions

def clean_captions(captions):
    table = str.maketrans('', '', string.punctuation)
    for img, caps in captions.items():
        captions[img] = [clean_caption(cap, table) for cap in caps]
    return captions

def clean_caption(caption, table):
    caption = caption.replace("-", " ").lower()
    tokens = caption.split()
    tokens = [token.translate(table) for token in tokens if token.isalpha()]
    return ' '.join(tokens)

def build_vocabulary(descriptions):
    vocab = set()
    for key in descriptions:
        for desc in descriptions[key]:
            vocab.update(desc.split())
    return vocab

def save_descriptions(descriptions, filename):
    lines = [f"{key}\t{desc}" for key, descs in descriptions.items() for desc in descs]
    with open(filename, "w") as file:
        file.write('\n'.join(lines))

dataset_text = r"C:\Users\Dell\Desktop\Academics\Spring 24\CSE 575 -  Statistical Machine Learning\Project"
dataset_images = r"C:\Users\Dell\Desktop\Academics\Spring 24\CSE 575 -  Statistical Machine Learning\Project\temp"
filename = f"{dataset_text}/Flickr8k.token.txt"

descriptions = extract_captions(filename)
print("Number of descriptions:", len(descriptions))

cleaned_descriptions = clean_captions(descriptions)
vocabulary = build_vocabulary(cleaned_descriptions)
print("Vocabulary size:", len(vocabulary))

save_descriptions(cleaned_descriptions, "descriptions.txt")

Number of descriptions: 8092
Vocabulary size: 8425


### Extract the Feature Vector

In [5]:
import os
from PIL import Image
import numpy as np
from tensorflow.keras.applications import Xception
from tqdm import tqdm
from pickle import dump

def extract_features(directory):
    model = Xception(include_top=False, pooling='avg')
    features = {}
    for pic in tqdm(os.listdir(directory)):
        file_path = os.path.join(directory, pic)
        if os.path.isfile(file_path):  # Check if the path is a file
            image = Image.open(file_path).resize((299, 299))
            image = np.expand_dims(image, axis=0)
            image = (image / 127.5) - 1.0
            feature = model.predict(image)
            features[pic] = feature
    return features

dataset_images = r"C:\Users\Dell\Desktop\Academics\Spring 24\CSE 575 -  Statistical Machine Learning\Project\Images"
features_file = "features.p"

# Extracting features
features = extract_features(dataset_images)
dump(features, open(features_file, "wb"))

# Loading features from the pickle file
features = load(open(features_file, "rb"))

  0%|                                                                                         | 0/8091 [00:00<?, ?it/s]

1/1 [==============================] - 1s 994ms/step


  0%|                                                                               | 1/8091 [00:01<2:23:24,  1.06s/it]

1/1 [==============================] - 0s 181ms/step


  0%|                                                                               | 2/8091 [00:01<1:18:39,  1.71it/s]

...

100%|█████████████████████████████████████████████████████████████████████████████▉| 8089/8091 [58:49<00:00,  2.45it/s]

1/1 [==============================] - 0s 311ms/step


100%|█████████████████████████████████████████████████████████████████████████████▉| 8090/8091 [58:49<00:00,  2.45it/s]

1/1 [==============================] - 0s 301ms/step


100%|██████████████████████████████████████████████████████████████████████████████| 8091/8091 [58:49<00:00,  2.29it/s]


### Loading dataset for model training

In [4]:
def load_photos(filename):
    with open(filename, 'r') as file:
        photos = file.read().split("\n")[:-1]
    return photos

def load_clean_descriptions(filename, photos):
    descriptions = {}
    with open(filename, 'r') as file:
        for line in file.read().split("\n"):
            words = line.split()
            if len(words) < 1:
                continue
            image, image_caption = words[0], words[1:]
            if image in photos:
                if image not in descriptions:
                    descriptions[image] = []
                desc = ' ' + " ".join(image_caption) + ' '
                descriptions[image].append(desc)
    return descriptions

def load_features(photos):
    all_features = load(open("features.p", "rb"))
    features = {k: all_features[k] for k in photos}
    return features

dataset_text = r"C:\Users\Dell\Desktop\Academics\Spring 24\CSE 575 -  Statistical Machine Learning\Project"
filename = f"{dataset_text}/Flickr_8k.trainImages.txt"

train_imgs = load_photos(filename)
train_descriptions = load_clean_descriptions("descriptions.txt", train_imgs)
train_features = load_features(train_imgs)

KeyError: '2513260012_03d33305cf.jpg'

### Tokenizing the Vocabulary

In [ ]:
from keras.preprocessing.text import Tokenizer
from pickle import dump

def dict_to_list(descriptions):
    all_desc = []
    for key in descriptions:
        all_desc.extend(descriptions[key])
    return all_desc

def create_tokenizer(descriptions):
    desc_list = dict_to_list(descriptions)
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(desc_list)
    return tokenizer

# Convert dictionary to a clear list of descriptions
def dict_to_list(descriptions):
    all_desc = []
    for key in descriptions.keys():
        all_desc.extend(descriptions[key])
    return all_desc

# Creating tokenizer class to vectorize text corpus
# Each integer will represent a token in the dictionary
def create_tokenizer(descriptions):
    desc_list = dict_to_list(descriptions)
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(desc_list)
    return tokenizer

# Save tokenizer to tokenizer.p pickle file
tokenizer = create_tokenizer(train_descriptions)
dump(tokenizer, open('tokenizer.p', 'wb'))

vocab_size = len(tokenizer.word_index) + 1
print("Vocabulary size:", vocab_size)

# Calculate maximum length of descriptions to decide the model structure parameters
def max_length(descriptions):
    desc_list = dict_to_list(descriptions)
    return max(len(d.split()) for d in desc_list)

max_length = max_length(descriptions)
print("Max length of description:", max_length)

### Create a Data generator

In [11]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

def data_generator(descriptions, features, tokenizer, max_length):
    while True:
        for key, description_list in descriptions.items():
            feature = features[key][0]
            inp_images, inp_seqs, op_words = create_sequences(tokenizer, max_length, description_list, feature)
            yield [[inp_images, inp_seqs], op_words]

def create_sequences(tokenizer, max_length, desc_list, feature):
    X1, X2, y = [], [], []
    for desc in desc_list:
        seq = tokenizer.texts_to_sequences([desc])[0]
        for i in range(1, len(seq)):
            in_seq, out_seq = seq[:i], seq[i]
            in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
            out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
            X1.append(feature)
            X2.append(in_seq)
            y.append(out_seq)
    return np.array(X1), np.array(X2), np.array(y)

train_generator = data_generator(train_descriptions, features, tokenizer, max_length)
sample_inputs, sample_output = next(train_generator)
sample_inputs[0].shape, sample_inputs[1].shape, sample_output.shape
# Output: ((47, 2048), (47, 32), (47, 7577))

KeyError: '1000268201_693b08cb0e.jpg'

### Define the CNN-RNN model

In [ ]:
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input, Dense, Dropout, LSTM, Embedding, add

def define_model(vocab_size, max_length):
    # Features from the CNN model compressed from 2048 to 256 nodes
    inputs1 = Input(shape=(2048,))
    fe1 = Dropout(0.5)(inputs1)
    fe2 = Dense(256, activation='relu')(fe1)

    # LSTM sequence model
    inputs2 = Input(shape=(max_length,))
    se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
    se2 = Dropout(0.5)(se1)
    se3 = LSTM(256)(se2)

    # Merging both models
    decoder1 = add([fe2, se3])
    decoder2 = Dense(256, activation='relu')(decoder1)
    outputs = Dense(vocab_size, activation='softmax')(decoder2)

    # Merge it [image, seq] [word]
    model = Model(inputs=[inputs1, inputs2], outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer='adam')

    # Summarize model
    model.summary()

    # Visualize model architecture
    plot_model(model, to_file='model.png', show_shapes=True)

    return model

### Training the Image Caption Generator model

In [ ]:
!pip install pydot

In [ ]:
!pip install graphviz

In [ ]:
import os
from keras.callbacks import ModelCheckpoint

# Training setup
print('Dataset:', len(train_imgs))
print('Descriptions (train):', len(train_descriptions))
print('Photos (train):', len(train_features))
print('Vocabulary Size:', vocab_size)
print('Description Length:', max_length)

# Define and compile the model
model = define_model(vocab_size, max_length)
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Directory to save models
os.makedirs("models", exist_ok=True)

# Training loop
epochs = 10
steps_per_epoch = len(train_descriptions)
checkpoint_path = "models/model_{epoch}.h5"

# Model checkpoint callback
checkpoint = ModelCheckpoint(checkpoint_path, monitor='loss', verbose=1, save_best_only=False, mode='min')

# Train the model for multiple epochs
for epoch in range(epochs):
    generator = data_generator(train_descriptions, train_features, tokenizer, max_length)
    model.fit(generator, epochs=1, steps_per_epoch=steps_per_epoch, callbacks=[checkpoint], verbose=1)


### Testing the Image Caption Generator model

In [ ]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from pickle import load
from collections import defaultdict

def extract_image_features(filename, model):
    try:
        image = Image.open(filename)
    except:
        print("ERROR: Can't open image! Ensure that image path and extension are correct")
        return None
    image = image.resize((299,299))
    image = np.array(image)
    if image.shape[2] == 4:  # Convert 4 channels images to 3 channels
        image = image[..., :3]
    image = np.expand_dims(image, axis=0)
    image = (image / 127.5) - 1.0
    feature = model.predict(image)
    return feature

def word_for_id(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None

def generate_description(model, tokenizer, photo, max_length):
    in_text = 'start'
    for _ in range(max_length):
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        sequence = pad_sequences([sequence], maxlen=max_length)
        pred = model.predict([photo, sequence], verbose=0)
        pred = np.argmax(pred)
        word = word_for_id(pred, tokenizer)
        if word is None or word == 'end':
            break
        in_text += ' ' + word
    return in_text

# Load the list of test image filenames
test_images_file = r"C:\Users\Dell\Desktop\Academics\Spring 24\CSE 575 -  Statistical Machine Learning\Project\Flickr_8k.testImages.txt"
with open(test_images_file, "r") as file:
    test_image_filenames = file.readlines()
test_image_filenames = [filename.strip() for filename in test_image_filenames]

# Load the tokenizer and the model
tokenizer = load(open("tokenizer.p", "rb"))
model = load_model("models/model_1.h5")
xception_model = Xception(include_top=False, pooling="avg")  # Assuming Xception model is defined

# Prepare a dictionary to store generated descriptions for each image
generated_descriptions = defaultdict(list)

# Iterate and display for each of the first 50 test images
for img_filename in test_image_filenames[:50]:
    img_id = img_filename.split('.')[0]  # Extract image ID (without the file extension)
    img_path = os.path.join(r"C:\Users\Dell\Desktop\Academics\Spring 24\CSE 575 -  Statistical Machine Learning\Project\Images", img_filename)
    
    photo = extract_image_features(img_path, xception_model)
    if photo is None:  # If the image couldn't be processed, skip to the next one
        continue
    description = generate_description(model, tokenizer, photo, 36)  # max_length = 36
    
    # Open and display the image
    image = Image.open(img_path)
    plt.imshow(image)
    plt.axis('off')  # Hide the axis
    plt.show()

    print(f"Image ID: {img_id}")
    print("Generated Description:", description)
    print()